In [1]:
import json
import random
import pandas as pd
import pymysql
from faker import Faker
from datetime import datetime, timedelta,time 
from dateutil.relativedelta import relativedelta  # Import relativedelta for date calculations
from dateutil.parser import parse  # Import parse for date string to datetime conversion
import pymysql

In [16]:
# Replace with your actual database connection details

connection = pymysql.connect(
    host='localhost',
    user='root',  # enter your username
    password='',  # enter your password 
    database='banking_system'
)

In [3]:
# Read the JSON data from the file
with open('C:/Users/tttie/OneDrive/Documents/CS 257/simulatedData/personData.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


## Person Table

In [4]:
person_df = pd.DataFrame(data)
#person_df.reset_index(level=0, inplace=True)
person_df['Address'] = person_df[['address', 'city', 'region', 'postalZip',  'country']].apply(
    lambda row: ', '.join(row.values.astype(str)), axis=1)

# Drop the individual address-related columns
person_df = person_df.drop(['address', 'city', 'postalZip', 'region', 'country'], axis=1)
person_df.head()

,PersonID,Name,DOB,Email,PhoneNumber,Address
0,1,Tatyana Fuentes,"Oct 23, 1999",id.nunc@yahoo.org,1-241-933-8368,"P.O. Box 231, 7475 Eleifend Ave, Santa Luzia, ..."
1,2,Cameran Bentley,"Jul 7, 2000",cras.eget@protonmail.edu,1-245-829-0562,"3728 Metus. Road, Magadan, Kentucky, 50904, Un..."
2,3,Xenos Noel,"Feb 16, 2000",facilisis@google.ca,1-451-353-7675,"Ap #126-2009 Ornare St., Vienna, Wisconsin, 27..."
3,4,Armando Ferguson,"Apr 3, 1991",magna@google.com,1-288-500-4115,"591-7472 Consectetuer Ave, Kroonstad, Missouri..."
4,5,Victor Spears,"Aug 21, 2000",nunc.quis.arcu@protonmail.net,1-258-725-3651,"457-9152 Ultrices Rd., San Luis Potosí, Iowa, ..."


## Account Table

In [6]:
fake = Faker()
# Define the start and end times for DateOpen
start_time = time(8, 0)  # 8:00 AM
end_time = time(17, 0)   # 5:00 PM
# Define a function to generate random dates within a given range
def random_date(start_date, end_date):
    if start_date > end_date:
        return end_date  # If start_date is after end_date, return end_date
    return start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))

# ...
accounts=[]
# Loop through each person in the DataFrame
for index, person in person_df.iterrows():
    # Convert the 'DOB' string to a datetime object
    birthday = parse(person['DOB'])

    # Calculate the minimum DateOpen based on birthday + 18 years
    minimum_date_open = birthday + relativedelta(years=18)

    # Determine how many accounts each person should have (you can modify this logic)
    num_accounts = random.randint(1, 2)  # Each person can have 1 or 2 accounts

    for _ in range(num_accounts):
        # Generate a random time within the allowed range (8:00 AM to 5:00 PM)
        random_time = time(random.randint(start_time.hour, end_time.hour), random.randint(0, 59))
        
        # Generate a random DateOpen by selecting a random day within the allowed range and combining it with the random time
        random_day = random_date(minimum_date_open, datetime(2023, 1, 1))
        date_open = datetime(
            random_day.year,
            random_day.month,
            random_day.day, 
            random_time.hour, random_time.minute, 0
        )

        accounts.append({
            "AccountID": len(accounts) + 1,  # Assign a unique AccountID
            "AccountNumber": fake.random_int(min=10000000, max=99999999),
            "AccountType": random.choice(['Savings', 'Checking']),
            "CurrentBalance": round(random.uniform(100, 10000), 2),
            "InterestRate": round(random.uniform(0.01, 0.1), 4),
            "DateOpen": date_open.strftime('%Y-%m-%d %H:%M:%S'),  # Include date and time
            "DateClose": None,  # Assume accounts are open
            "AccountStatus": 'Active',
            "PersonID": person['PersonID']  # Use the PersonID from the DataFrame
        })

# The 'accounts' list now contains multiple account records for each person with DateOpen >= birthday + 18 years, including date and time within the specified range (8:00 AM to 5:00 PM)
accounts_df = pd.DataFrame(accounts)
accounts_df.head()

,AccountID,AccountNumber,AccountType,CurrentBalance,InterestRate,DateOpen,DateClose,AccountStatus,PersonID
0,1,71639742,Checking,6213.79,0.0398,2021-05-14 16:08:00,None,Active,1
1,2,43136785,Checking,3292.66,0.0702,2022-07-13 16:51:00,None,Active,2
2,3,45564058,Savings,1717.53,0.0663,2021-08-27 11:32:00,None,Active,2
3,4,82307862,Checking,9406.93,0.0690,2022-01-24 15:21:00,None,Active,3
4,5,49939750,Savings,5825.06,0.0972,2009-07-22 17:42:00,None,Active,4


## Branch Table

In [7]:
branch_data = [
    {
        "BranchID": 1,
        "BranchName": "CS1",
        "Address": "123 Dickson Street, San Jose CA 95132, United States",
        "PhoneNumber": "1-927-288-228"
    },
    {
        "BranchID": 2,
        "BranchName": "CS2",
        "Address": "5121 Adipiscing Rd, Seatle  Washington 89765, United States",
        "PhoneNumber": "1-555-402-403"
    },
    {
        "BranchID": 3,
        "BranchName": "CS3",
        "Address": "89 Creekside Ave, Austin Texas 78934, United States",
        "PhoneNumber": "1-953-456-1234"
    },
    {
        "BranchID": 4,
        "BranchName": "CS4",
        "Address": "457-6683 Curabitur Ave, Irvine CA 89456, United States",
        "PhoneNumber": "1-222-455-445"
    },
    {
        "BranchID": 5,
        "BranchName": "CS5",
        "Address": "8501 Carnegie Bvld, New York, New York, United States",
        "PhoneNumber": "1-606-909-808"
    }
]
branch_df = pd.DataFrame(branch_data)

## Employee Table

In [8]:


# Create a list of roles and the corresponding number of employees for each role
roles = [
    {"Role": "Manager", "Count": 1},
    {"Role": "Teller", "Count": 3},
    {"Role": "Accountant", "Count": 2},
    {"Role": "Customer Service", "Count": 2},
    {"Role": "Assistant Manager", "Count": 1}
]

# Create an empty list to store employee records
employees = []

# Initialize PersonIDs
person_ids = list(range(1, 501))

# Shuffle the PersonIDs to randomize assignments
random.shuffle(person_ids)

# Assign employees to 5 branches with the specified roles
branch_count = 5  # Number of branches
for branch_id in range(1, branch_count + 1):
    for role in roles:
        for _ in range(role["Count"]):
            if person_ids:
                person_id = person_ids.pop()
                employees.append({
                    "EmployeeID": len(employees) + 1,
                    "Position": role["Role"],
                    "PersonID": person_id,
                    "BranchID": branch_id
                })

# Create a DataFrame of employees
employees_df = pd.DataFrame(employees)

# Display the employee DataFrame
employees_df

,EmployeeID,Position,PersonID,BranchID
0,1,Manager,145,1
1,2,Teller,57,1
2,3,Teller,485,1
3,4,Teller,171,1
4,5,Accountant,255,1
5,6,Accountant,267,1
6,7,Customer Service,388,1
7,8,Customer Service,283,1
8,9,Assistant Manager,90,1
9,10,Manager,499,2


### Save data


In [9]:
person_df.to_json('person_data.json', orient='records')
accounts_df.to_json('account_data.json', orient='records')
branch_df.to_json('branch_data.json', orient='records')
employees_df.to_json('employees_data.json', orient='records')


In [ ]:
'''
cursor = connection.cursor()

# Execute a SELECT statement
cursor.execute("SELECT * FROM your_table")
result = cursor.fetchall()

# Print the query result
for row in result:
    print(row)

cursor.close()
connection.close()
'''

## Account Table

In [10]:
with open('C:/Users/tttie/OneDrive/Documents/CS 257/simulatedData/account_data.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)


In [30]:
accounts_df = pd.DataFrame(data)
accounts_df.head(10)

,AccountID,AccountNumber,AccountType,CurrentBalance,InterestRate,DateOpen,DateClose,AccountStatus,PersonID
0,1,71639742,Checking,6213.79,0.0398,2021-05-14 16:08:00,None,Active,1
1,2,43136785,Checking,3292.66,0.0702,2022-07-13 16:51:00,None,Active,2
2,3,45564058,Savings,1717.53,0.0663,2021-08-27 11:32:00,None,Active,2
3,4,82307862,Checking,9406.93,0.0690,2022-01-24 15:21:00,None,Active,3
4,5,49939750,Savings,5825.06,0.0972,2009-07-22 17:42:00,None,Active,4
5,6,85634081,Checking,7738.06,0.0738,2017-09-17 16:32:00,None,Active,4
6,7,43138536,Checking,321.82,0.0125,2022-05-09 15:55:00,None,Active,5
7,8,73661915,Savings,4546.76,0.0150,2022-01-11 11:38:00,None,Active,5
8,9,53492911,Checking,8554.48,0.0860,2009-12-28 09:09:00,None,Active,6
9,10,83287285,Checking,9676.74,0.0392,2017-07-13 16:22:00,None,Active,6


In [27]:
len(accounts_df[accounts_df['PersonID']==5])

2

In [17]:
accounts_df[accounts_df['PersonID']==2]['AccountID'].tolist()

[2, 3]

In [22]:
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta

transactions = []

def random_date(start_date, end_date):
    return start_date + relativedelta(
        hours=random.randint(8, 16),  # Random hour between 8 AM and 4 PM
        minutes=random.randint(0, 59),  # Random minute
    )

for index, account in accounts_df.iterrows():
    date_open = datetime.strptime(account['DateOpen'], '%Y-%m-%d %H:%M:%S')
    minimum_transaction_date = date_open
    num_transactions = random.randint(1, 10)
    
    transaction_date = minimum_transaction_date  # Initialize transaction_date
    
    for _ in range(num_transactions):
        transaction_type = random.choice(['Deposit', 'Withdrawal', 'Transfer'])
        
        # Check if a person can make a transfer
        person_accounts = accounts_df[accounts_df['PersonID'] == account['PersonID']]
        
        if len(person_accounts) == 1 and transaction_type == 'Transfer':
            continue  # Skip transfer for individuals with only one account
        
        if transaction_type == 'Withdrawal':
            max_withdrawal_amount = account['CurrentBalance']
            amount = round(random.uniform(10, max_withdrawal_amount), 2)
            transferID = None
        elif transaction_type == 'Deposit':
            amount = round(random.uniform(1, 2000), 2)
            transferID = None
        else:  # Transaction is a transfer
            # Ensure that the person has more than one account for a transfer
            if len(person_accounts) < 2:
                continue  # Skip transfer for individuals with only one account

            # Select another account for the transfer
            transfer_accounts = person_accounts[person_accounts['AccountID'] != account['AccountID']]
            transfer_account = transfer_accounts.sample(n=1)
            transferID = transfer_account['AccountID'].values[0]
            
            # Ensure that the transaction dates are increasing for each person
            if transaction_date > minimum_transaction_date:
                minimum_transaction_date = transaction_date
            else:
                transaction_date = minimum_transaction_date

        # Generate a random transaction date
        transaction_date = random_date(transaction_date, datetime(2023, 10, 1))

        transaction = {
            "TransactionID": len(transactions) + 1,
            "AccountID": account['AccountID'],
            "TransactionType": transaction_type,
            "Amount": amount,
            "TransactionDate": transaction_date.strftime('%Y-%m-%d %H:%M:%S'),
            "TransferAccountID": transferID
        }
        transactions.append(transaction)


In [24]:
transactions_df = pd.DataFrame(transactions)
transactions_df

,TransactionID,AccountID,TransactionType,Amount,TransactionDate,TransferAccountID
0,1,1,Deposit,1219.97,2021-05-15 04:04:00,NaN
1,2,1,Deposit,1756.65,2021-05-15 12:18:00,NaN
2,3,2,Deposit,652.35,2022-07-14 05:05:00,NaN
3,4,2,Deposit,1306.30,2022-07-14 17:19:00,NaN
4,5,2,Withdrawal,146.54,2022-07-15 04:57:00,NaN
...,...,...,...,...,...,...
3839,3840,760,Deposit,900.28,2022-02-09 06:19:00,NaN
3840,3841,760,Transfer,900.28,2022-02-09 22:17:00,759.0
3841,3842,760,Deposit,1803.68,2022-02-10 07:51:00,NaN
3842,3843,761,Deposit,1952.08,2012-02-08 23:51:00,NaN


In [28]:
transactions_df.head(30)

,TransactionID,AccountID,TransactionType,Amount,TransactionDate,TransferAccountID
0,1,1,Deposit,1219.97,2021-05-15 04:04:00,NaN
1,2,1,Deposit,1756.65,2021-05-15 12:18:00,NaN
2,3,2,Deposit,652.35,2022-07-14 05:05:00,NaN
3,4,2,Deposit,1306.30,2022-07-14 17:19:00,NaN
4,5,2,Withdrawal,146.54,2022-07-15 04:57:00,NaN
5,6,2,Transfer,146.54,2022-07-15 18:00:00,3.0
6,7,2,Transfer,146.54,2022-07-16 10:06:00,3.0
7,8,2,Withdrawal,64.95,2022-07-16 23:00:00,NaN
8,9,3,Deposit,358.54,2021-08-28 02:56:00,NaN
9,10,3,Deposit,610.70,2021-08-28 16:20:00,NaN


In [29]:
transactions_df.to_json('transactions_data.json', orient='records')
